In [28]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
nifty = "^NSEI" 
df = yf.download(nifty, start="2022-01-01", end="2024-12-31", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [29]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
Date,,,,,
2022-01-03,17625.699219,17646.650391,17383.300781,17387.150391,200500
2022-01-04,17805.250000,17827.599609,17593.550781,17681.400391,247400
2022-01-05,17925.250000,17944.699219,17748.849609,17820.099609,251500
2022-01-06,17745.900391,17797.949219,17655.550781,17768.500000,236500
2022-01-07,17812.699219,17905.000000,17704.550781,17797.599609,239300


In [30]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 738 entries, 2022-01-03 to 2024-12-30
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (Close, ^NSEI)   738 non-null    float64
 1   (High, ^NSEI)    738 non-null    float64
 2   (Low, ^NSEI)     738 non-null    float64
 3   (Open, ^NSEI)    738 non-null    float64
 4   (Volume, ^NSEI)  738 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 34.6 KB


Price,Close,High,Low,Open,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
count,738.000000,738.000000,738.000000,738.000000,7.380000e+02
mean,19869.030026,19965.183202,19765.920797,19877.742619,2.852400e+05
std,2861.194916,2870.945028,2856.172270,2864.353296,9.259101e+04
min,15293.500000,15382.500000,15183.400391,15272.650391,0.000000e+00
25%,17606.987305,17686.899902,17488.974609,17610.662598,2.303000e+05
50%,18914.674805,19026.474609,18849.599609,18918.450195,2.662000e+05
75%,22217.749512,22364.812988,22080.799805,22244.437012,3.234000e+05
max,26216.050781,26277.349609,26151.400391,26248.250000,1.006100e+06


In [31]:
df.index.name = 'Date'
df = df[['Open', 'High', 'Low', 'Close', 'Volume']] 
df.head()


Price,Open,High,Low,Close,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
Date,,,,,
2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,200500
2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,247400
2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,251500
2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,236500
2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,239300


In [32]:
def calculate_bb(data,multiplier):
  data['SMA']=data['Close'].rolling(window=20).mean()
  data['SD']=data['Close'].rolling(window=20).std()
  data['Upper_Band']=data['SMA']+multiplier*data['SD']
  data['Lower_Band']=data['SMA']-multiplier*data['SD']
  
  return data

def add_vwap(df):
    df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
    return df

def macd(df):
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['signalline'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def stochastic(df, k_window=14, d_window=3):
    low_min = df['Low'].rolling(window=k_window).min()
    high_max = df['High'].rolling(window=k_window).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(window=d_window).mean()
    return df

def atr(df, window=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    tr = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = tr.rolling(window=window).mean()
    return df

# Add all indicators
df = calculate_bb(df,2)
df = add_vwap(df)
df = macd(df)
df = rsi(df)
df = stochastic(df)
df = atr(df)

In [33]:
print(df.head())
df.to_csv("DATA.csv")

Price               Open          High           Low         Close  Volume  \
Ticker             ^NSEI         ^NSEI         ^NSEI         ^NSEI   ^NSEI   
Date                                                                         
2022-01-03  17387.150391  17646.650391  17383.300781  17625.699219  200500   
2022-01-04  17681.400391  17827.599609  17593.550781  17805.250000  247400   
2022-01-05  17820.099609  17944.699219  17748.849609  17925.250000  251500   
2022-01-06  17768.500000  17797.949219  17655.550781  17745.900391  236500   
2022-01-07  17797.599609  17905.000000  17704.550781  17812.699219  239300   

Price      SMA  SD Upper_Band Lower_Band          VWAP  ...         EMA26  \
Ticker                                                  ...                 
Date                                                    ...                 
2022-01-03 NaN NaN        NaN        NaN  17551.883464  ...  17625.699219   
2022-01-04 NaN NaN        NaN        NaN  17656.969085  ...  17638.